# Single Player Game Demonstration

Scythe is a fantastic board game. If you are looking to get schooled, ask Helge for a game. The game takes place on a map, which looks like that

![](map.png)

We will be the green player, who starts with workers at `m1` and `f1`. For the single player game workers are important, we can ignore everything else. The last important thing are the available actions, which are described in the player mat fof rthe industrial player

![](industrial.jpg)

Finally some basic imports, and starting the game as the green industrial player.

In [1]:
import re
import random
from kosa import Client
client = Client()

client.join_a_game(color='green', player_mat='INDUSTRIAL')
client.start()
print('game id ', client.game_id)

player_id  a7df59ad-94bc-42de-ab21-427e77b614d7
joined game with id  670ec42b-e126-4b24-83d0-40ef432098a2
join OK
start OK
game id  670ec42b-e126-4b24-83d0-40ef432098a2


We now perform a mind blowing set of actions.

In [2]:
import json

def perform_action_matching(action, regex):
    available_actions = client.get_available_actions()
    if available_actions and action in available_actions:
        option_list = client.get_available_options(action)
        index_to_play = None
        for idx, val in enumerate(option_list):
            result = re.compile(regex).findall(json.dumps(val))
            if len(result) > 0:
                index_to_play = idx
                break
    
        if index_to_play is not None:
            client.perform_action(action, index_to_play)
    
for _ in range(16):
    perform_action_matching('MOVE', 'GainCoin')
    perform_action_matching('TRADE', 'Popularity')
    perform_action_matching('MOVE', 'GainCoin')
    perform_action_matching('BOLSTER', 'BolsterPower')
    perform_action_matching('PRODUCE', 'm1')
    
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('TRADE', 'TradePopularity')
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('TRADE', 'TradePopularity')

We have now obtained 2 stars, which we can check as follows:

In [3]:
def get_star_events():
    game = json.loads(client.export_game() or '[]')
    return [g for g in game if g['type'] == 'StarEvent']

def print_stars():
    print("Stars: " , [s['star'] for s in get_star_events()])
    
print_stars()

Stars:  ['MAX_POWER', 'MAX_POPULARITY']


# Moving around
The following performs some moves to place workers.

In [4]:
perform_action_matching('MOVE','WORKER_2.*v1')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'GainCoin')
perform_action_matching('PRODUCE','VILLAGE')

In [5]:
perform_action_matching('MOVE', 'WORKER_1.*f1.*FARM')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*t2.*TUNDRA')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*m3.*MOUNTAIN')
perform_action_matching('PRODUCE','VILLAGE')
perform_action_matching('MOVE', 'WORKER_1.*w3.*FOREST')
perform_action_matching('PRODUCE','VILLAGE')

In [6]:
print_stars()

Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']


In [7]:
perform_action_matching('MOVE', 'WORKER_7.*f1.*FARM')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_7.*t2.*TUNDRA')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_7.*m3.*MOUNYAIN')
perform_action_matching('PRODUCE','MOUNTAIN')
perform_action_matching('MOVE', 'WORKER_7.*v5.*VILLAGE')
perform_action_matching('PRODUCE','MOUNTAIN')

In [8]:
perform_action_matching('MOVE', 'WORKER_4.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_4.*m1.*MOUNTAIN')
perform_action_matching('PRODUCE','MOUNTAIN')

In [9]:
perform_action_matching('MOVE', 'WORKER_3.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_3.*t2.*TUNDRA')
perform_action_matching('PRODUCE','FOREST')

In [10]:
perform_action_matching('MOVE', 'WORKER_8.*f1.*FARM')
perform_action_matching('PRODUCE','FOREST')
perform_action_matching('MOVE', 'WORKER_8.*t2.*TUNDRA')
perform_action_matching('PRODUCE','TUNDRA')
perform_action_matching('MOVE', 'WORKER_8.*m3.*MOUNTAIN')
perform_action_matching('PRODUCE','TUNDRA')

# Perform secondary action to gain more stars

Currently the game finished event is not captured correctly. So, we get an exception. One can see the game finishing, by watching `nc localhost 1337`.

In [11]:
def perform_seconday_action_if_available():
    available_actions = client.get_available_actions()
    if available_actions:
        for action in ['ENLIST', 'DEPLOY', 'BUILD', 'UPGRADE']:
            if action in available_actions:
                number = len(client.get_available_options(action) or [])
                if number > 0:
                    client.perform_action(action, random.randint(0, number))
            
while not client.is_game_over():
    perform_action_matching('BOLSTER', '')
    perform_seconday_action_if_available()
    
    perform_action_matching('TRADE', 'Popularity')
    perform_seconday_action_if_available()
        
    perform_action_matching('MOVE', 'GainCoin')
    perform_seconday_action_if_available()

    perform_action_matching('PRODUCE', random.choice(['MOUNTAIN', 'FOREST', 'TUNDRA']))
    perform_seconday_action_if_available()
    perform_action_matching('TRADE', 'OIL')
    perform_seconday_action_if_available()
    
    print_stars()

Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WORKERS', 'ALL_RECRUITS']
Stars:  ['MAX_POWER', 'MAX_POPULARITY', 'ALL_WO

In [12]:
client.get_stats()